In [1]:
#!pip install pyTelegramBotAPI

In [17]:
import telebot
import time
import requests
import datetime
from PIL import Image
import pickle

In [3]:
msg = None

In [2]:
url_ocr = 'http://127.0.0.1:8000/upload'
url_predict = 'http://127.0.0.1:8000/upload'  

In [ ]:
bot = telebot.TeleBot('5838296860:AAG8UTY2WVmQ2TDGX6LjWN65cZTYCOGabGI')


@bot.message_handler(commands=["start"])
def handle_start(message):
    bot.send_message(message.from_user.id, "Отправьте, пожалуйста картинку")


@bot.message_handler(content_types=["text", "photo"])
def handle_photo(message):
    chat_id = message.from_user.id
    print('in', chat_id, message.text)
    #print(message)
    #global msg
    #msg = message
    if message.photo is None:
        bot.send_message(message.from_user.id, "Отправьте, пожалуйста картинку")   
    else:
        #print(len(message.photo))
        fileID = message.photo[-1].file_id   
        file_info = bot.get_file(fileID)
        

        print(file_info.file_path)
        
        downloaded_file = bot.download_file(file_info.file_path)
        filename = datetime.datetime.now().strftime("%d.%m.%Y_%H.%M.%S.%f")
        print(filename)
        with open(f'{filename}.png', 'wb') as f:
            f.write(downloaded_file)
        print('saved')
        with open(f'{filename}.png', 'rb') as f:
            resp = requests.post(url=url_ocr, files={'file': f})
        
        text = resp.json()['message']
        print(text)
        img = Image.open(f'{filename}.png')
        
        with open(f'{filename}.pkl', 'wb') as f:
            pickle.dump((img, text), f)

        with open(f'{filename}.pkl', 'rb') as f:
            resp = requests.post(url=url_predict, files={'file': f})
        result = f'{text}\n{resp.json()["message"]}'
        
        print(resp.json()["message"])
        
        bot.send_message(message.from_user.id, result)
             

    #print('out', chat_id, message.text)
    
    
bot.infinity_polling()

in 192193316 None
photos/file_0.jpg
20.12.2022_00.46.48.126988
saved
Идеального кольца не суще: .
in 192193316 None
photos/file_4.jpg
20.12.2022_00.50.10.490180
saved
Когда-нибудь я приду на свадьбу своих детей, откажусь есть то, что они подадут, и потребую наггетсы и чипсы: 16.23
